In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법_with_markdown.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

194

In [3]:
document_list[46]

Document(metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공

In [6]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
    #model="solar-embedding-1-large-passage"
)

In [5]:
len(embedding.embed_query("소득세법"))

4096

In [7]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index-markdown"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 7.56 s
Wall time: 53.6 s


# 2. 답변 생성 전 retrieval 확인

In [8]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [11]:
retrieved_docs[1]

Document(id='49fe3bcb-2b2b-43db-9f79-3ebb6e78353b', metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 

In [12]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [13]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [14]:
retrived_docs[1]

Document(id='49fe3bcb-2b2b-43db-9f79-3ebb6e78353b', metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 

# 3. 답변 생성

In [15]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {
            "role":"user",
            "content": f"""- [context]를 참고해서 사용자의 질문에 10줄이내로 답변해 주세요
- [context] : {retrieved_doc}
- 질문 : {query}"""
        }
    ],
    temperature=0.2
)

In [16]:
print(response.choices[0].message.content)

연봉이 5천만원인 직장인의 소득세를 계산하기 위해 먼저 종합소득과세표준에 따른 세율을 적용해야 합니다.

1. **종합소득 과세표준**: 5천만원
2. **세율 적용**: 5천만원 초과 8,800만원 이하 구간에 해당하므로, 
   - 기본세액: 624만원
   - 초과금액: 5천만원 초과 금액은 5천만원 - 5천만원 = 0원이므로 추가 세액 없음.

따라서, 종합소득산출세액은 624만원입니다.

3. **근로소득세액공제**: 총급여액이 3천 300만원 초과 5천만원 이하이므로,
   - 공제액: 74만원 - [(5천만원 - 3천 300만원) × 8/1000] = 74만원 - 13.6만원 = 60.4만원 (단, 66만원보다 적지 않으므로 66만원 적용)

4. **최종 소득세액**: 624만원 - 66만원 = 558만원

따라서, 연봉이 5천만원인 직장인의 소득세는 약 558만원입니다.


In [17]:
from openai import OpenAI
import os
# upstage의 과금정책 : https://www.upstage.ai/pricing/api
client = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role":"system", "content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {
            "role":"user",
            "content": f"""- [context]를 참고해서 사용자의 질문에 5줄이내로 답변해 주세요
- [context] : {retrieved_doc}
- 질문 : {query}"""
        }
    ],
    temperature=0.2
)

In [18]:
print(response.choices[0].message.content)

연봉 5천만원의 근로소득세액은 다음과 같이 계산됩니다:  

1. **종합소득산출세액**:  
   - 1,400만원 이하: 1,400만원 × 6% = 84만원  
   - 1,400만원 초과 5,000만원 이하: (5,000만원 - 1,400만원) × 15% + 84만원 = **624만원**  

2. **근로소득세액공제 적용**:  
   - 총급여액 3,300만원 초과 7,000만원 이하 구간:  
     **74만원 - [(5,000만원 - 3,300만원) × 8/1,000] = 74만원 - 12만원 = 62만원**  
     (단, 최소 66만원 적용 → **66만원**)  

3. **최종 산출세액**:  
   - 624만원(산출세액) - 66만원(공제) = **558만원**  

※ 단, 추가 공제(자녀세액공제 등) 및 지방소득세(10%)는 별도 적용 필요합니다.


In [24]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [19]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [20]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [21]:
ai_message = llm.invoke(prompt)

In [22]:
print(ai_message.content)

연봉 5,000만원인 직장인의 소득세는 다음과 같이 계산됩니다:

---

### **1. 종합소득산출세액 계산**
- **과세표준**: 총급여액 5,000만원 (기타 소득 및 공제 미고려, 단순 계산)
- **세율 적용** (제55조 제1항):
  - 1,400만원 이하: 6% → 84만원  
  - 1,400만원 초과 5,000만원 이하: 84만원 + (5,000만원 - 1,400만원) × 15%  
    = **84만원 + 3,600만원 × 15% = 84만원 + 540만원 = 624만원**

---

### **2. 근로소득세액공제 적용** (제59조 제1항, 제2항)
- **총급여액**: 5,000만원 (3,300만원 초과 7,000만원 이하 구간)
- **공제액 계산**:
  ```
  74만원 - (5,000만원 - 3,300만원) × 8/1,000  
  = 74만원 - 1,700만원 × 0.008  
  = 74만원 - 13.6만원 = 60.4만원
  ```
- **최소 공제액**: 66만원 (계산 결과 60.4만원이므로 66만원 적용)

---

### **3. 최종 소득세 계산**
- **종합소득산출세액**: 624만원  
- **근로소득세액공제**: 66만원  
- **최종 세액**:  
  ```
  624만원 - 66만원 = **558만원**
  ```

---

### **4. 추가 고려 사항**
- **자녀세액공제**: 질문자 정보에 자녀 관련 내용이 없으므로 미적용.  
- **일용근로자 공제**: 일반 직장인은 해당되지 않음.  
- **기타 공제**: 사회보장료, 보험료 등 추가 공제는 고려하지 않았습니다.

---

### **결론**
연봉 5,000만원인 직장인의 **소득세는 약 558만원**입니다.  
(단, 실제 세금은 보험료, 의료비 등 추가 공제를 반영할 경우 달라질 수 있습니다.)


# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조

In [26]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model = "gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large")

# 2. # 업로드한 벡터db를 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])

In [27]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원의 경우 총급여액이 3,300만원 초과 7,000만원 이하에 해당하므로, 근로소득 세액공제는 74만원 - [(5,000만원 - 3,300만원) × 8/1,000] = 62.8만원입니다. 단, 66만원 미만이므로 최종 공제액은 66만원입니다.  \n\n종합소득 산출세액은 제55조 세율을 적용해야 하지만, 제공된 정보만으로는 정확한 소득세 계산이 불가능합니다. (추가 정보 필요)'

# 5. 키워드 사전 활용

In [28]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [30]:
vec1 = embedding.embed_query("총급여")
vec2 = embedding.embed_query("연봉")
print("총급여와 연봉의 유사도 :", cosine_similarity(vec1, vec2))

총급여와 연봉의 유사도 : 0.6874992849327983


In [31]:
vec1 = embedding.embed_query("종합소득")
vec2 = embedding.embed_query("연봉")
print("총급여와 연봉의 유사도 :", cosine_similarity(vec1, vec2))

총급여와 연봉의 유사도 : 0.5662459197662915


In [32]:
vec1 = embedding.embed_query("직장인")
vec2 = embedding.embed_query("거주자")
print("총급여와 연봉의 유사도 :", cosine_similarity(vec1, vec2))

총급여와 연봉의 유사도 : 0.5402446852505834


In [34]:
vec1 = embedding.embed_query("5천만원")
vec2 = embedding.embed_query("5,000만원")
print("총급여와 연봉의 유사도 :", cosine_similarity(vec1, vec2))

총급여와 연봉의 유사도 : 0.9122784065935228


In [35]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = vectorstore.similarity_search(query, k=2)
retrieved_docs

[Document(id='b5d33d4f-34b0-4fe1-93ce-035c8cedbb2f', metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n\n제59조의2(자녀세액공제) ①종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀(입양자 및 위탁아동을 포함하며, 이하 이 조에서 “공제대상자녀”라 한다) 및 손자녀로서 8세 이상의 사람에 대해서는 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다. <개정 2015. 5. 13., 2017. 12. 19., 2018. 12. 31., 2019. 12. 31., 2022. 12. 31., 2023. 12. 31., 2024. 12. 31.>\n\n1. 1명인 경우: 연 25만원\n\n2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양

In [36]:
query = "연봉 5000만원인 거주자의 소득세는 얼마인가요?"
retrieved_docs = vectorstore.similarity_search(query, k=2)
retrieved_docs

[Document(id='49fe3bcb-2b2b-43db-9f79-3ebb6e78353b', metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12.

In [66]:
# 사람을 나타내는 표현 -> 거주자로 변경 / 5천만원->5000만원
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
llm = ChatOpenAI(model="gpt-4o-mini")
dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 
변경해 주세요. 만약 변경할 필요가 없을 경우, 사용자의 질문을 변경하지 않아도 됩니다.
질문만 리턴해 주세요
사전 : {dictionary}
질문 : {{question}}""")

In [67]:
parser = StrOutputParser()
parser.invoke(llm.invoke(prompt.invoke({"question":"소득이 높은 남자가 있습니다"})))

'소득이 높은 거주자가 있습니다.'

In [68]:
dictionary_chain = prompt | llm | StrOutputParser()
dictionary_chain.invoke({"question":"소득이 높은 남자가 5천만원 가지고 있습니다"})

'소득이 높은 거주자가 5천만원 가지고 있습니다.'

In [69]:
dictionary_chain.invoke("연봉 5천만원인 직장인의 소득세는 얼마예요?")

'연봉 5천만원인 거주자의 소득세는 얼마예요?'

In [70]:
# rag_chain.invoke('연봉 5천만원인 거주자의 소득세는 얼마예요?')
final_chain = dictionary_chain | rag_chain

In [71]:
final_chain.invoke("연봉 5천만원인 직장인의 소득세는 얼마예요?")

'연봉 5천만원(과세표준 기준)에 대한 소득세는 다음과 같이 계산됩니다:  \n\n1. **근로소득공제 적용**: 연봉 5천만원에서 근로소득 공제(약 1,275만원)를 제외한 과세표준은 약 3,725만원입니다(제47조).  \n2. **종합소득산출세액 계산**: 3,725만원에 대해 제55조 세율(1,400만원 이하 6%, 초과 15%)을 적용하면, 산출세액은 **384만원**입니다.  \n3. **세액공제 적용**: 추가 공제(자녀세액공제 등)가 없을 경우 최종 세액은 **384만원**입니다.  \n\n단, 주어진 문맥은 "총급여액 기준" 특별공제 규정을 제시하나, 이는 근로소득 원천징수 시 적용되는 방식으로, 정확한 세액 계산에는 근로소득 공제와 종합소득세율(제55조)이 반영되어야 합니다.'

In [ ]:
# 입력 {"question":"질문"} -> dictionary_chain이 질문 멘트를 개선
# -> 개선된 질문을 rag_chain에 전달 -> retriever가 관련 문서 검색 -> format_documents
# -> 완성된 prompt를 llm에 전달하여 답변 생성 -> 문자만 추출해서 최종 답변